# Symbolic Schemes

As of version 0.10, *findiff* can also provide a symbolic representation
of finite difference schemes suitable for using in conjunction with *sympy*.
The main use case is to facilitate deriving your own iteration schemes.

As an example, we will derive the simple iteration scheme for a Jacobi
solver for the 2-dimensional Poisson equation

$$
\nabla^2 u = \rho
$$

First of all, make some imports from *findiff* and *sympy*, and create a symbolic mesh:

In [52]:
from findiff import SymbolicMesh, SymbolicDiff
from sympy import symbols, Eq, solve

mesh = SymbolicMesh("x, y")
dx, dy = mesh.spacing
u = mesh.create_symbol("u")
rho = mesh.create_symbol(r"\rho")
m, n = symbols("m, n")

Meshed symbols are symbols that can carry any number of symbolic indices:

In [53]:
u[m, n]

u[m, n]

In [54]:
rho[m, n]

\rho[m, n]

Next, create symbolic representations of the differential operators in finite difference approximation. To execute them on the mesh functions, simply call them:

In [56]:
d2_dx2, d2_dy2 = [SymbolicDiff(mesh, axis=k, degree=2) for k in range(2)]

(
    d2_dx2(u, at=(m, n), offsets=(-1, 0, 1)) + 
    d2_dy2(u, at=(m, n), offsets=(-1, 0, 1))
)

(u[m, n + 1] + u[m, n - 1] - 2*u[m, n])/\Delta y**2 + (u[m + 1, n] + u[m - 1, n] - 2*u[m, n])/\Delta x**2

In [65]:
from sympy import latex
print(latex(_56))

\frac{{u}_{m,n + 1} + {u}_{m,n - 1} - 2 {u}_{m,n}}{\Delta y^{2}} + \frac{{u}_{m + 1,n} + {u}_{m - 1,n} - 2 {u}_{m,n}}{\Delta x^{2}}


The parameter `at` is the index tuple of the mesh point where the derivative is to be evaluated. The `offsets` parameter specifies which neighboring points you want your approximation to use. Using `-1, 0, 1` gives the usual 2nd-order accurate, symmetric approximation for the second derivate. This works for all grid points except at the boundary, of course. That's not problem for iteration schemes if you have Dirichlet boundary conditions, which we will assume here. Otherwise, you would have to create separate representations for the boundary using offsets like `[0, 1, 2, 3]`.

Anyways, let's continue our derivation:

In [57]:
Eq(_, rho[m,n])

Eq((u[m, n + 1] + u[m, n - 1] - 2*u[m, n])/\Delta y**2 + (u[m + 1, n] + u[m - 1, n] - 2*u[m, n])/\Delta x**2, \rho[m, n])

Solve it for $u_{m,n}$:

In [58]:
solve(_, u[m,n])

[(-\Delta x**2*\Delta y**2*\rho[m, n] + \Delta x**2*u[m, n + 1] + \Delta x**2*u[m, n - 1] + \Delta y**2*u[m + 1, n] + \Delta y**2*u[m - 1, n])/(2*(\Delta x**2 + \Delta y**2))]

In [59]:
_[0]

(-\Delta x**2*\Delta y**2*\rho[m, n] + \Delta x**2*u[m, n + 1] + \Delta x**2*u[m, n - 1] + \Delta y**2*u[m + 1, n] + \Delta y**2*u[m - 1, n])/(2*(\Delta x**2 + \Delta y**2))

Let's assume that we have the same grid spacing along both axes:

In [60]:
h = symbols("h")
_.subs({dx: h, dy: h})

(-h**4*\rho[m, n] + h**2*u[m + 1, n] + h**2*u[m - 1, n] + h**2*u[m, n + 1] + h**2*u[m, n - 1])/(4*h**2)

In [61]:
_.simplify()

-h**2*\rho[m, n]/4 + u[m + 1, n]/4 + u[m - 1, n]/4 + u[m, n + 1]/4 + u[m, n - 1]/4

So we finally have our Jacobi iteration scheme:

In [62]:
Eq(u[m, n], _)

Eq(u[m, n], -h**2*\rho[m, n]/4 + u[m + 1, n]/4 + u[m - 1, n]/4 + u[m, n + 1]/4 + u[m, n - 1]/4)